<a href="https://colab.research.google.com/github/urmidedhia/Synapse-Learning-Period/blob/NLP-week-2/Latent_Dirichlet_Allocation_Urmi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Latent Dirichlet Allocation ##
**Watch these videos:**
- https://www.youtube.com/watch?v=azXCzI57Yfc
- https://www.youtube.com/watch?v=T05t-SqKArY
- https://www.youtube.com/watch?v=BaM1uiCpj_E

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import numpy as np
import pandas as pd
import sklearn

In [35]:
'''
Load the dataset from the CSV and save it to 'data_text'. We only need the'headline_text' column,
so drop the rest of the columns. Call the final dataframe "documents" 
Then, add an 'index' column to the dataframe. Be sure to name it index.
Hint: dataframe.index
'''
data_text_path =  '/content/drive/MyDrive/Week 2 NLP (Week 7)/abcnews-date-text.csv'
data_text = pd.read_csv(data_text_path)
documents = data_text.drop('publish_date', axis = 1)
documents['index'] = documents.index
documents

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
...,...,...
1103660,the ashes smiths warners near miss liven up bo...,1103660
1103661,timelapse: brisbanes new year fireworks,1103661
1103662,what 2017 meant to the kids of australia,1103662
1103663,what the papodopoulos meeting may mean for ausus,1103663


Let's glance at the dataset:

In [36]:
'''
print the total number of documents
'''
print(len(documents))

1103665


In [37]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.

This time, we'll make use of the gensim library and show you how  to preprocess text easily. 

In [38]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [39]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [40]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [41]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [42]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    '''
        1) Lemmatize the text
        2) Then, stem the text
        3) return final text
    '''
    wnl = WordNetLemmatizer()
    lem_word = wnl.lemmatize(text)
    ps = PorterStemmer()
    lem_stemmed_word = ps.stem(lem_word) 
    return lem_stemmed_word


# Tokenize and lemmatize. Look up how preprocessing is being done
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            # TODO: Apply lemmatize_stemming on the token, then add to the results list
    return result

In [43]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]
print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes

In [44]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = []
processed_docs = documents['headline_text'].map(preprocess)
# for text in documents['headline_text'].values:
#   processed_docs.append(preprocess(text))

In [45]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [14]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
#  Todo
dictionary = gensim.corpora.Dictionary(processed_docs)

In [15]:
'''
Checking dictionary created
'''
print(dictionary)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

Dictionary(62458 unique tokens: ['broadcast', 'commun', 'decid', 'licenc', 'awar']...)
0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [16]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
- keep 100000 most frequent tokens
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned above
dictionary.filter_extremes(no_below = 15, no_above = 0.1, keep_n = 100000)
print(dictionary)

Dictionary(14280 unique tokens: ['broadcast', 'commun', 'decid', 'licenc', 'awar']...)


** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [17]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'.
Hint: Since we want a BoW model for each document, you can use list comprehension instead of for loops. List
comprehension is the more pythonic way.
'''
#  Todo:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [18]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(76, 1), (112, 1), (486, 1), (4050, 1)]

In [19]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 486 ("rain") appears 1 time.
Word 4050 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [20]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models
# TODO
tfidf = models.TfidfModel(bow_corpus)

In [21]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
# TODO
corpus_tfidf = [tfidf[i] for i in bow_corpus]


In [22]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903416018683463),
 (1, 0.385232906171445),
 (2, 0.4974398602008866),
 (3, 0.5056144643792737)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [23]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'

If Multicore is throwing errors on you laptop, use LDA mono-core, gensim.models.LdaModel
'''
# TODO
lda_model = models.LdaMulticore(corpus=bow_corpus, num_topics=10, id2word = dictionary, chunksize=20000, passes=2, workers = 2)

In [24]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.019*"year" + 0.017*"queensland" + 0.016*"market" + 0.014*"australian" + 0.012*"farmer" + 0.012*"deal" + 0.012*"share" + 0.009*"price" + 0.009*"drug" + 0.008*"research"


Topic: 1 
Words: 0.017*"rural" + 0.016*"miss" + 0.014*"council" + 0.014*"rise" + 0.012*"busi" + 0.012*"rate" + 0.011*"news" + 0.010*"budget" + 0.009*"live" + 0.009*"search"


Topic: 2 
Words: 0.024*"govern" + 0.022*"crash" + 0.021*"elect" + 0.017*"say" + 0.014*"dy" + 0.013*"chang" + 0.012*"labor" + 0.010*"support" + 0.009*"hill" + 0.009*"defend"


Topic: 3 
Words: 0.017*"world" + 0.016*"australia" + 0.013*"open" + 0.013*"final" + 0.012*"trump" + 0.011*"win" + 0.008*"record" + 0.007*"leagu" + 0.007*"lead" + 0.007*"australian"


Topic: 4 
Words: 0.036*"interview" + 0.026*"child" + 0.023*"countri" + 0.020*"hour" + 0.012*"port" + 0.011*"victim" + 0.010*"abus" + 0.010*"royal" + 0.009*"john" + 0.009*"speak"


Topic: 5 
Words: 0.031*"polic" + 0.022*"death" + 0.020*"court" + 0.019*"coast" + 0.016*"south" + 0

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: Australian markets
* 1: Rural budget
* 2: Government 
* 3: Australian sports
* 4: Child abuse
* 5: Murder investigation
* 6: Health
* 7: Brisbane attacks
* 8: Science
* 9: Murder trial

## Step 4.2 Running LDA using TF-IDF ##

In [25]:
'''
Define lda model using corpus_tfidf
'''
# TODO
lda_model_tfidf = models.LdaMulticore(corpus=corpus_tfidf, num_topics=10, id2word = dictionary, chunksize=20000, passes=2, workers = 2)

In [26]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.010*"kill" + 0.009*"grandstand" + 0.005*"bomb" + 0.005*"octob" + 0.005*"attack" + 0.005*"dead" + 0.005*"august" + 0.005*"australian" + 0.004*"blast" + 0.004*"soldier"


Topic: 1 Word: 0.010*"weather" + 0.008*"interview" + 0.006*"decemb" + 0.006*"jame" + 0.005*"sexual" + 0.005*"peter" + 0.005*"spring" + 0.005*"april" + 0.005*"cyclon" + 0.004*"alic"


Topic: 2 Word: 0.016*"polic" + 0.016*"charg" + 0.013*"murder" + 0.012*"court" + 0.011*"woman" + 0.010*"crash" + 0.009*"jail" + 0.009*"death" + 0.008*"miss" + 0.008*"accus"


Topic: 3 Word: 0.017*"countri" + 0.016*"hour" + 0.008*"podcast" + 0.008*"drum" + 0.005*"labor" + 0.005*"elect" + 0.005*"govern" + 0.004*"live" + 0.004*"liber" + 0.004*"industri"


Topic: 4 Word: 0.008*"farmer" + 0.006*"interview" + 0.006*"rain" + 0.006*"dairi" + 0.005*"drought" + 0.005*"price" + 0.005*"domest" + 0.005*"flood" + 0.005*"violenc" + 0.005*"water"


Topic: 5 Word: 0.012*"news" + 0.012*"rural" + 0.007*"nation" + 0.007*"juli" + 0.007*"south" +

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: Bomb attack
* 1: Annual weather report
* 2: Car crash trial
* 3: Government
* 4: Agricultural issues
* 5: Weekly weather report
* 6: Business and currency
* 7: Sports
* 8: Rugby league
* 9: Country budget

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model

We will check to see where our test document would be classified. 

In [27]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [28]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.467724472284317	 
Topic: 0.031*"polic" + 0.022*"death" + 0.020*"court" + 0.019*"coast" + 0.016*"south" + 0.015*"melbourn" + 0.013*"investig" + 0.013*"gold" + 0.013*"accus" + 0.012*"north"

Score: 0.37226229906082153	 
Topic: 0.018*"health" + 0.011*"china" + 0.011*"need" + 0.010*"servic" + 0.009*"report" + 0.008*"cost" + 0.008*"sale" + 0.008*"flood" + 0.008*"train" + 0.008*"build"

Score: 0.02000579610466957	 
Topic: 0.019*"year" + 0.017*"queensland" + 0.016*"market" + 0.014*"australian" + 0.012*"farmer" + 0.012*"deal" + 0.012*"share" + 0.009*"price" + 0.009*"drug" + 0.008*"research"

Score: 0.02000274509191513	 
Topic: 0.017*"rural" + 0.016*"miss" + 0.014*"council" + 0.014*"rise" + 0.012*"busi" + 0.012*"rate" + 0.011*"news" + 0.010*"budget" + 0.009*"live" + 0.009*"search"

Score: 0.020001795142889023	 
Topic: 0.036*"interview" + 0.026*"child" + 0.023*"countri" + 0.020*"hour" + 0.012*"port" + 0.011*"victim" + 0.010*"abus" + 0.010*"royal" + 0.009*"john" + 0.009*"speak"

Score: 

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model

In [31]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
Code would be similar to above
'''
# Todo
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8199699521064758	 
Topic: 0.036*"interview" + 0.026*"child" + 0.023*"countri" + 0.020*"hour" + 0.012*"port" + 0.011*"victim" + 0.010*"abus" + 0.010*"royal" + 0.009*"john" + 0.009*"speak"

Score: 0.020014794543385506	 
Topic: 0.031*"polic" + 0.022*"death" + 0.020*"court" + 0.019*"coast" + 0.016*"south" + 0.015*"melbourn" + 0.013*"investig" + 0.013*"gold" + 0.013*"accus" + 0.012*"north"

Score: 0.020002814009785652	 
Topic: 0.017*"rural" + 0.016*"miss" + 0.014*"council" + 0.014*"rise" + 0.012*"busi" + 0.012*"rate" + 0.011*"news" + 0.010*"budget" + 0.009*"live" + 0.009*"search"

Score: 0.0200024526566267	 
Topic: 0.018*"health" + 0.011*"china" + 0.011*"need" + 0.010*"servic" + 0.009*"report" + 0.008*"cost" + 0.008*"sale" + 0.008*"flood" + 0.008*"train" + 0.008*"build"

Score: 0.020002344623208046	 
Topic: 0.041*"charg" + 0.023*"face" + 0.022*"school" + 0.021*"hous" + 0.021*"plan" + 0.017*"murder" + 0.016*"trial" + 0.015*"worker" + 0.013*"court" + 0.012*"peopl"

Score: 0.02000233

## Step 6: Testing model on unseen document ##

In [32]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.417361319065094	 Topic: 0.017*"world" + 0.016*"australia" + 0.013*"open" + 0.013*"final" + 0.012*"trump"
Score: 0.22362878918647766	 Topic: 0.024*"govern" + 0.022*"crash" + 0.021*"elect" + 0.017*"say" + 0.014*"dy"
Score: 0.21899457275867462	 Topic: 0.018*"health" + 0.011*"china" + 0.011*"need" + 0.010*"servic" + 0.009*"report"
Score: 0.020007450133562088	 Topic: 0.031*"polic" + 0.022*"death" + 0.020*"court" + 0.019*"coast" + 0.016*"south"
Score: 0.02000357024371624	 Topic: 0.041*"charg" + 0.023*"face" + 0.022*"school" + 0.021*"hous" + 0.021*"plan"
Score: 0.02000315859913826	 Topic: 0.017*"rural" + 0.016*"miss" + 0.014*"council" + 0.014*"rise" + 0.012*"busi"
Score: 0.020000893622636795	 Topic: 0.036*"interview" + 0.026*"child" + 0.023*"countri" + 0.020*"hour" + 0.012*"port"
Score: 0.0200001560151577	 Topic: 0.034*"sydney" + 0.024*"polic" + 0.011*"futur" + 0.011*"prison" + 0.010*"darwin"
Score: 0.020000087097287178	 Topic: 0.019*"year" + 0.017*"queensland" + 0.016*"market" + 0.0

### Now, look up different ways we can transform our words or use different techniques or improvements that we can make to make our Topic Classifier more accurate. Write them down below:

**We can:**
- Append points here
- Use or create a stop word list in accordance to our corpus.
- Replace numerical values with '#num#' to reduce vocabulary size if need be.
- Use LDAboost.
